In [1]:
from dotenv import load_dotenv
from huggingface_hub import login
import os
load_dotenv()
token = os.getenv('HUGGINGFACE_HUB_TOKEN')
login(token=token)

/opt/anaconda3/envs/textSum/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
os.chdir('..')

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    repo_id: Path
    local_dir_use_symlinks: bool

In [4]:
from textSummarizer.constant import *
from textSummarizer.utils.common import read_yaml, create_directories

In [5]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath: Path = CONFIG_FILE_PATH,
        params_filepath: Path = PARAMS_FILE_PATH,
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        data_ingestion_config = DataIngestionConfig(
            root_dir = Path(config.root_dir),
            repo_id = config.repo_id,
            local_dir_use_symlinks = config.local_dir_use_symlinks,
        )

        return data_ingestion_config

In [6]:
import os
from textSummarizer.logging import logger
from huggingface_hub import snapshot_download


In [7]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_data(self):
    # Create parent directory if it doesn't exist
        os.makedirs(os.path.dirname(self.config.root_dir), exist_ok=True)

        if not os.path.exists(self.config.root_dir) or not os.listdir(self.config.root_dir):
            logger.info("Starting data download...")
            snapshot_download(
                repo_id = self.config.repo_id,
                repo_type = "dataset",
                local_dir = self.config.root_dir,
                local_dir_use_symlinks = self.config.local_dir_use_symlinks,
            )
            
            logger.info(
                f"Data downloaded successfully!"
            )
        else:
            logger.info("Files already exists. Skipping download.")

In [8]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_data()
except Exception as e:
    raise e

[2026-01-31 12:57:37,171: INFO: common: yaml file: config/config.yaml loaded successfully]
[2026-01-31 12:57:37,176: INFO: common: yaml file: params.yaml loaded successfully]
[2026-01-31 12:57:37,178: INFO: common: Created directory at: artifacts]
[2026-01-31 12:57:37,179: INFO: 3524136610: Starting data download...]


/opt/anaconda3/envs/textSum/lib/python3.11/site-packages/huggingface_hub/file_download.py:979: UserWarning: `local_dir_use_symlinks` parameter is deprecated and will be ignored. The process to download files to a local folder has been updated and do not rely on symlinks anymore. You only need to pass a destination folder as`local_dir`.
For more details, check out https://huggingface.co/docs/huggingface_hub/main/en/guides/download#download-files-to-local-folder.
  warnings.warn(
Fetching 5 files: 100%|██████████| 5/5 [00:00<00:00,  9.18it/s]

[2026-01-31 12:57:37,991: INFO: 3524136610: Data downloaded successfully!]
